In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
from pathlib import Path
import os, sys, datetime, time, random, fnmatch, math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import skimage.metrics

import torch
from torchvision import transforms as tvtransforms
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.utils as vutils
import torch.utils.tensorboard as tensorboard
import torch.nn as nn

import datasets, transforms

# Input Directories
data_BSE = "D:/data/JSRT/BSE_JSRT"
data_normal = "D:/data/JSRT/JSRT"

# Image Size:
image_spatial_size = (440,440)

test_length = 10

# Weight Initialisation
def weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight.data, 0., 0.02)
        try:
            nn.init.constant_(m.bias.data, 0.)
        except:
            pass
    if isinstance(m, nn.BatchNorm2d):
        if m.affine:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0.)

## Code for putting things on the GPU
ngpu = 2
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)

cpu


In [2]:
# Current date:
current_date=datetime.datetime.today().strftime('%Y-%m-%d')


# Data Loader

discriminator_keys_images = ["source", "boneless"]
ds = datasets.JSRT_CXR(data_normal, data_BSE,
                         transform=tvtransforms.Compose([
                             transforms.CLAHE(discriminator_keys_images),
                             transforms.ZScoreNormalisation(discriminator_keys_images),
                             transforms.RescalingNormalisation(discriminator_keys_images,(0,1)),
                             transforms.RandomHorizontalFlip(discriminator_keys_images, probability=0.5),
                             transforms.RandomVerticalFlip(discriminator_keys_images, probability=0.5),
                             transforms.IntensityJitter(discriminator_keys_images,source_image_key="source", rescale_factor_limits=(0.75,1.0), window_motion_limits=(-1,1)),
                             transforms.RandomIntensityComplement(discriminator_keys_images, probability=0.5),
                             transforms.RandomRotation(discriminator_keys_images),
                             transforms.Rescale(image_spatial_size, discriminator_keys_images, None),
                             transforms.ToTensor(discriminator_keys_images),
                             ])
                      )
"""
                             
                             """
plt.imshow(ds[16]["source"][0,:])
plt.show
lengths=(len(ds)-test_length, test_length)
ds_training, ds_testing = torch.utils.data.random_split(ds, lengths)
ds

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …